In [ ]:
# Basic Packages
import numpy as np
import h5py
import logging
import os
import shutil
import gc
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import linregress, pearsonr, spearmanr
import seaborn as sns
from scipy.optimize import curve_fit
import scipy.stats as stats
import math
# Physics-related Packages
from astropy.cosmology import Planck15
from sklearn.mixture import GaussianMixture
from sklearn.metrics import pairwise_distances
from astropy.cosmology import Planck15 as cosmo
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture
import pickle

In [ ]:
with open('/users_path/merger_trace/data/tng_cluster/tng_cluster_products/feats_labels_dict_tngcluster.pkl', 'rb') as f:
    feats_labels_dict = pickle.load(f)

In [ ]:
mass_ratio_vec = []
for halo_id in list(feats_labels_dict.keys()):
    #print(halo_id)
    for snap in range(72, 100):
        snap_dict = feats_labels_dict[halo_id].get(snap, {})
        if 'mass_ratio' in snap_dict:
            mass_ratio = snap_dict['mass_ratio']
            mass_ratio_vec.append(mass_ratio)
            #print(f"Snap {snap} halo {halo_id}: mass_ratio = {mass_ratio}")
        else:
            print(f"Snap {snap} halo {halo_id}: 'mass_ratio' not found")


In [ ]:
mass_ratio_vec = np.array(mass_ratio_vec)

In [ ]:
plt.figure(figsize=(6, 4))
plt.hist(mass_ratio_vec, bins=30, color='skyblue', edgecolor='black')
plt.xlabel('Mass Ratio')
plt.ylabel('Count')
plt.title('Histogram of Mass Ratios')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
cat_name_300 = '/users_path/merger_trace/data/tng300/tng300_targetcat/TargetHaloCat_092_TNG300/TargetHalo_MergerCat_092.hdf5'

In [ ]:
with h5py.File(cat_name_300) as cat:
    FOF_Halo_IDs = cat['Group/FOF_Halo_IDs'][:]
    Group_M_Crit200 = cat['Group/Group_M_Crit200'][:]
    SubhaloGrNr = cat['Subhalo/SubhaloGrNr'][:]
    SubhaloMass = cat['Subhalo/SubhaloMass'][:]

target_ids = FOF_Halo_IDs[Group_M_Crit200>=0.7*1e+3]
mass_ratios = []

for halo_id in target_ids:
    Subhalo_in = np.where(SubhaloGrNr==halo_id)[0]
    SubhaloMass_in = SubhaloMass[Subhalo_in]
    
    if len(SubhaloMass_in)>=2:
        sorted_indices = np.argsort(SubhaloMass_in)[::-1]  

        # find the most massive and seond massive
        mass1 = SubhaloMass_in[sorted_indices[0]]
        mass2 = SubhaloMass_in[sorted_indices[1]]
        mass_ratio = mass2/mass1
    elif len(SubhaloMass_in)==1:
        mass_ratio = 0
    else:
        mass_ratio = -1
    
    mass_ratios.append(mass_ratio)

    print(f'finish mass ratio {halo_id}')

In [ ]:
def get_mass_ratio(cat_name, halo_id):
    with h5py.File(cat_name) as cat:
        Halo_IDs = cat['Group/FOF_Halo_IDs'][:]
        GroupPos = cat['Group/GroupPos'][:]
        Group_R_Crit200 = cat['Group/Group_R_Crit200'][:]
        
        SubhaloGrNr = cat['Subhalo/SubhaloGrNr'][:]
        SubhaloMasses = cat['Subhalo/SubhaloMass'][:]
        SubhaloPos = cat['Subhalo/SubhaloPos'][:]

    halo_pos =  GroupPos[Halo_IDs == halo_id][0]
    halo_x, halo_y, halo_z = halo_pos[0], halo_pos[1], halo_pos[2]
    halo_r = Group_R_Crit200[Halo_IDs==halo_id]

    Subhalo_in = np.where(
        (SubhaloGrNr == halo_id) &
        (SubhaloPos[:,0] <= halo_x + halo_r) &
        (SubhaloPos[:,0] >= halo_x - halo_r) &
        (SubhaloPos[:,1] <= halo_y + halo_r) &
        (SubhaloPos[:,1] >= halo_y - halo_r) &
        (SubhaloPos[:,2] <= halo_z + 2*halo_r) &
        (SubhaloPos[:,2] >= halo_z - 2*halo_r) 
    )[0]

    SubhaloMass_in = SubhaloMasses[Subhalo_in]
    sorted_masses = np.sort(SubhaloMass_in)[::-1] 

    if len(sorted_masses) >= 2:
        m1, m2 = sorted_masses[0], sorted_masses[1]
        top_sorted = np.argsort(SubhaloMass_in)[::-1]
        top_indices = Subhalo_in[top_sorted[:2]] 
    elif len(sorted_masses) == 1:
        m1, m2 = sorted_masses[0], 0
        top_indices = [Subhalo_in[0]]
    else:
        m1, m2 = 0, 0
        top_indices = []
    

    top_positions = SubhaloPos[top_indices]  
    
    relative_xy = top_positions[:, :2] - halo_pos[:2]
    pixel_coords = (relative_xy / (2*halo_r) * 240).astype(int) + 120  


    return m1, m2, m2 / m1 if m1 > 0 else np.nan, pixel_coords

In [ ]:
plt.figure(figsize=(8, 5))
plt.hist(mass_ratios, bins=30, color='skyblue', edgecolor='black')
plt.xlabel('Mass Ratio')
plt.ylabel('Count')
plt.title('Histogram of Mass Ratios')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
mass_ratios = np.array(mass_ratios)
target_ids[mass_ratios>=0.2]

In [ ]:
selected_mass_ratios = []
for halo_id in target_ids:
     _, _, selected_mass_ratio, _ =get_mass_ratio(cat_name_300, halo_id)
     selected_mass_ratios.append(selected_mass_ratio)

In [ ]:
plt.figure(figsize=(8, 5))
plt.hist(selected_mass_ratios, bins=30, color='skyblue', edgecolor='black')
plt.xlabel('Mass Ratio')
plt.ylabel('Count')
plt.title('Histogram of Mass Ratios')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
target_ids[np.array(selected_mass_ratios)>=0.2]

In [ ]:
np.array(selected_mass_ratios)[target_ids==142]